In [ ]:
import os
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import re

import plotly.graph_objects as go
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)


---
# <center><b>1. 파일 불러오기</b></center>
---

In [ ]:
# 파일 경로 지정하기
os.chdir('C:\민원분석참조모델\data\결과')

In [ ]:
# 결과 폴더 파일 목록 가져오기
df_list = list(os.listdir(os.getcwd()))
df_list

In [ ]:
# 몇번째 파일 분석할건지 지정
index = 0

In [ ]:
# 분석할 파일 불러오기
df = pd.read_csv(df_list[index], encoding='euc-kr')

In [ ]:
# 파일 개요
df.info()

In [ ]:
# 필요한 컬럼 추출
df = df[['민원제목']]
df

---
# <center><b>2. 데이터 전처리</b></center>
---

In [ ]:
kind = list() # 민원제목유형 리스트
main = list() # 민원제목내용 리스트

# 정규식 이용해서 괄호 문자열 제거하기
regex = "\(.*\)|\s-\s.*|\[.*\]"
df = df.dropna() # 결측치 제거

for i in df.index:
    start = df['민원제목'][i].find('[')
    if start != -1:
        end = df['민원제목'][i].find(']', start)
        title = df['민원제목'][i][start:end+1].strip()
        script = df['민원제목'][i][end+1:].strip()
        script = re.sub(regex, '', script).replace('\n\n*', '')
    else:
        title = "유형없음"
        script = df['민원제목'][i]
        
    kind.append(title)
    main.append(script)

# 컬럼 추가하기
df['민원제목유형'] = kind
df['민원제목내용'] = main

df = df[['민원제목유형', '민원제목내용', '민원제목']]
df.to_csv(
    f'../../result/3.민원제목/민원제목_{df_list[index]}', encoding='euc-kr', index=False)


In [ ]:
# 민원제목유형에 따른 빈도 수
df_kind = df.groupby('민원제목유형', as_index=True).count()
df_kind = df_kind[['민원제목']]
df_kind = df_kind.rename(columns={'민원제목': '빈도'})
df_kind.sort_values(by='빈도', ascending=False, inplace=True)

df_kind.to_csv(f'../../result/3.민원제목/민원제목유형_빈도_{df_list[index]}', encoding='euc-kr')

---
# <center><b>3. 데이터 시각화</b></center>
---

In [ ]:
# 민원제목유형 빈도 파이차트

# 레이아웃 설정
layout = {
    'title': {
        'text': '<b>민원 제목유형 빈도 파이차트 분석</b>',
        'font': {
            'size': 20
        },
        'x': 0.5,
    },
    'showlegend': True,
    'legend': {
        'title': {
            'text': '<b>제목유형</b>',
            'font': {
                'size': 15
            },
        },
        'borderwidth': 20,
        'bordercolor': '#fff',
        # 'bgcolor': '#F0F8FF',
    },
    'autosize': True
    # 'width': 600,
    # 'height': 600,
}

fig = go.Figure()

fig.add_trace(
    go.Pie(
        labels=df_kind.index, values=df_kind['빈도']
    )
)
fig.update_traces(textposition='inside', textinfo='percent+label+value')

fig.update_layout(layout)

fig.show()


---
# <center><b>4. 워드 클라우드</b></center>
---

In [ ]:
# 워드클라우드 단어 모으기

word = set()
for i in df.index:
    temp = df['민원제목내용'][i].strip().split()
    for j in temp:
        word.add(j.replace(',', ''))

In [ ]:
# 워드 문자열화
word_str = ' '.join(word)

In [ ]:
# 불용어 처리
STOP_WORDS = {
    '신고', '신고입니다', '신고합니다',
    'quot', '1AA', 'apos', 'x29', 'x28',
    '단속해주세요', '하였습니다',
    'http', 'https', 'https www',
    '안되나요', '해주세요', '있습니다', '갑니다',
    '상황입니다', 'SPP', '에', '주',
    '하는데', '안녕하세요', '문의드립니다', '심합니다',
    '심하네요', '을', '를', '은', '는', '있다', '제발',
    '입니다', '바랍니다', '요청', '다', '봐주세요',
    '민원입니다', '대놓네요', '하네요', '사진', '아닌가요',
    '부탁드립니다', '부탁', '부탁합니다', '2AA', '으로', '에서',
    '가', '이', '구', '되나요', '이거', '했습니다', '로', '도',
    'S', '부', '겁니까', '봅니다', '같아요', '않아요', '않습니다',
    '확인부탁드려요', '부탁드려요', '아닙니다', '중입니다', '맞나요',
    '하세요', '처리해주세요', '요청드립니다', '있어요', '내용',
    '문의', '안녕하십니까', '주십시오', '그리고', '그래서',
    '빨리', '없네요', '오후', '오전', '오늘', '현재',
    '합니다', '앞입니다', '진짜', '등', '많습니다', '수고하세요', '수고하십니다',
    '있네요', '같습니다', '그런데', '있나요', '사진입니다', '없나요',
    '또', '어렵습니다', '습니다', '저희는', '는데', '어요', '부터', '지금',
    '위', '음', '됩니까', '라는', '고', '과',
    '앞', '옆', '있음', '같은', '납니다', '주세요', '없습니다', '생깁니다',
    '빨리치워주세요', '치워주세요',
    '감사하겠습니다', '조치부탁드립니다',
    '하나요', '드립니다', '특히', '불편하네요', '위험합니다',
    '동영상', '올립니다', '없어요', '주민입니다', '안됩니다',
    '놔두고', '줍니다', '아닙니까', '어제', '나네요', '및', '했네요',
    '매번', '상습적으로', '아닌데', '꼭', '아침', '상습',
    '수고많으십니다', '하더군요', '어떻게', '정말', 'ㅜㅜ', '^^',
    '합니까', '뭐하는', '하는', '계속', '아니면', '해주십시오', '해주십시요',
    'map', 'GmapShow', 'net service', 'B3 A0',
    'event php', 'BO EA', 'do position', 'go', 'kr', 'go kr', 'amp',
    'downloadFile do', 'It', 'www', 'www clien', 'gmap go',
    'service board', 'youtube', 'youtube com', 'naver com',
    'board hongbo', 'portalap map', 'blog', 'blog naver', 'www gmap',
    'kr portalap', 'GmapShow', 'do', 'GmapShow do', 'lur downloadFile',
    'ED BO', 'instagram com', 'do', 'com', 'com watch', 'watch v',
    'www ppeum20', 'EC AC', 'ppeum20 com', 'mimimi co', 'amp',
    'tistory com', 'br', 'gt', 'br gt', 'itemId vendorItemId',
    'portalap', 'clien net', 'tistory', 'naver', 'gmap',
    'position', 'service board', 'net service', 'net',
    'youtu be', 'smartsotre', 'NaPm ct', 'nbsp', 'clien',
    'm', 'It', 'it', 'lur downloadFile', 'instagram', 'vendorItemId q',
    'board hongbo', 'dscfNtNo', 'service', 'board', 'hongbo',
    '의', 'IT', 'me', 'coupang', 'A', 'downloadFile',
    'youtu', 'be', 'youtube', 'watch', 'v', 'V',
    '기관지정요청', '안내', '갤러리에서 불러와',
    '열어보실', '제거', 'URL', 'URL을 복사하여',
    '해당', '해당 URL을', '정보시스템', '안내', '지방행정통합',
    '지방행정통합 정보시스템', '신고위치 URL',
    '붙여넣으면', '열어보실 수', '갤러리', '에서', '불러와', '첨부한',
    '여기', '때문에', ' 불법주차', '불법주차 ', ' URL을 복사하여',
    '브라우저', '복사하여', '제가', '불법주차 불법주차',
    '아니고', '사람이', '다시', '더', '좀', '못하게', '많은',
    '담당자분들께서는 첨부파일', '바로 촬용하여',
    '부근', '저는', '스토리지에서 1개월 자료를 보관하고', '인해 스토리지에서', '공간으로 인해', '그냥', 'G', '너무', '있는데', '스토리지', '담당자분들께서는', '첨부파일',
    '그', '스토리이제어 1개월', '다른', '저장 공간으로', 'URL을',
    '사진촬영시간', '감사합니다', '너무', 'e',
    '경우', 'p', '밤', '야간', '건', '뒤', '앞', '요망', '신청',
    '네요', '없음', '않음', '국', '상',
    '못한다', '밤', '때', '관련', '위반내용', '문제', '추가',
    '진정', '신고파일', '파일', '민원', '사람', '발생'
}


In [ ]:
# 워드 클라우드 폰트 설정

FONT_PATH = 'C:/Users/user/AppData/Local/Microsoft/Windows/Fonts/AppleSDGothicNeoB.ttf'

In [ ]:
# 워드클라우드 분석

wordcloud = WordCloud(
    stopwords=STOP_WORDS,
    font_path=FONT_PATH,
    background_color='white',
    width=800,
    height=800).generate(word_str)

In [ ]:
# 워드클라우드 이미지 확인

plt.figure(figsize=(100, 100))  # 이미지 사이즈 지정
plt.imshow(wordcloud) 
plt.axis('off')  # x y 축 숫자 제거
plt.show()